This notebook should plot change in monthly runoff, for two time slices, for all 75 basins.

18 Mar 2024 | EHU

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import itertools
import json
import os


## Generic filepath to the main data folder -- downloaded Finn's and put all together
## no regional separation
fpath = '/Users/lizz/Documents/Research/Runoff-intercomparison/basin_aggregated/Runoff/all_regions/'

gmodels = ['GloGEM', 'PyGEM', 'OGGM']
#All of the climate models used
modelnames_short = ['BCC-CSM2-MR',
                    'MPI-ESM1-2-HR',
                    'MRI-ESM2-0',
                    'CESM2-WACCM',
                    'NorESM2-MM'] ## these are the ones for which we have GCM data as of Oct 2023

SSPpaths = ['ssp126','ssp245','ssp370','ssp585']   #Specifiying the SSP

In [ ]:
this_GCM = modelnames_short[0]
this_basin = 'INDUS'
scen=SSPpaths[1]

In [ ]:
## Basins in the order they are written
basins_all = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103','YANGTZE' : '2433'}

## Make an array to guarantee the order will remain intact
basin_names = basins_all.keys() ## keys actually remember insertion order as of Python 3.7, wow modern

## Compute relative change
We want the monthly mean at the end of the century minus the monthly mean in early century, divided by the early-century mean, to show relative change for each month.  This will reflect the net change in seasonality -- e.g. shifts of runoff away from melt peak month and towards earlier precip peak.

In [ ]:
import matplotlib.patches as mpatches


earlyC_start_year = 2000
earlyC_end_year = 2020 ## 1 after the true end, for < operator

lateC_start_year = 2081
lateC_end_year = 2101


example_colors_w = {'GloGEM': 'darkgreen',
                    'PyGEM': 'Purple',
                    'OGGM': 'royalblue'}

fig, axs = plt.subplots(15,5, figsize=(12,18), sharey=True, sharex=True)
for i,b in enumerate(basin_names):
    ax = axs.ravel()[i]
    ax.axhline(0, color='k', ls=':')

    this_basin=b
    
    multiGCM_multiGEM_df = {g: {m: [] for m in modelnames_short} for g in gmodels}

    for m in modelnames_short:
        fname = fpath +'runoff_AlignedMonthly_{}_{}_{}.csv'.format(m, scen, this_basin)
        temp_df = pd.read_csv(fname, index_col=0)
        temp_df.index = pd.to_datetime(temp_df.index)
        multiGCM_multiGEM_df['GloGEM'] = temp_df['GloGEM']
        multiGCM_multiGEM_df['PyGEM'] = temp_df['PyGEM']
        multiGCM_multiGEM_df['OGGM'] = temp_df['OGGM']        

    
    for g in gmodels:
        multiGCM_df_thisGEM = pd.DataFrame.from_dict(multiGCM_multiGEM_df[g])
        
        early_C_thisGEM = multiGCM_df_thisGEM.loc[(multiGCM_df_thisGEM.index.year>earlyC_start_year)
                              & (multiGCM_df_thisGEM.index.year<earlyC_end_year)]
        early_C_monthmeans = early_C_thisGEM.groupby(by=[early_C_thisGEM.index.month]).median()

        late_C_thisGEM = multiGCM_df_thisGEM.loc[(multiGCM_df_thisGEM.index.year>lateC_start_year)
                              & (multiGCM_df_thisGEM.index.year<lateC_end_year)]
        late_C_monthmeans = late_C_thisGEM.groupby(by=[late_C_thisGEM.index.month]).median()
        
        diff_monthly = early_C_monthmeans - late_C_monthmeans
        
        # if b=='DANUBE':
        #     print('early_C_monthmeans: {} \n late_C_monthmeans: {} \n diff_monthly: {}'.format(early_C_monthmeans, late_C_monthmeans, diff_monthly))
        
        # ax.plot((diff_monthly/early_C_monthmeans.sum()).median(axis=1), 
        #         color=example_colors_w[g], marker='o', label=g)
        
        markerlines, stemlines, baselines = ax.stem(np.arange(1,13), (diff_monthly/early_C_monthmeans.sum()).median(axis=1), 
                                                    linefmt='-', markerfmt='o', basefmt='k:')
        markerlines.set_markerfacecolor('none')
        plt.setp(stemlines, color=example_colors_w[g])
        plt.setp(markerlines, color=example_colors_w[g])
        
    # ax.tick_params(axis='x', labelrotation=45)
    ax.set(xticks=np.arange(1,13), xticklabels=['1','','','4','','', '7', '', '', '10', '', '12'],
          ylim=(-0.5,0.5),
          yticks=(-0.5, 0, 0.5), yticklabels=['-50%', '0', '+50%'])
    # # ax.set_xlabel('Years', fontsize=14)
    # # ax.set_ylabel('Rolling mean SPEI', fontsize=14)
    ax.text(0.05, 0.85, this_basin, transform=ax.transAxes) 
    # ax.set(title='{}, 3 glacier models'.format(example_b))
    # plt.tight_layout()
    
green_patch = mpatches.Patch(color=example_colors_w['GloGEM'], label='GloGEM')
purple_patch = mpatches.Patch(color=example_colors_w['PyGEM'], label='PyGEM') 
blue_patch = mpatches.Patch(color=example_colors_w['OGGM'], label='OGGM')
axs[0,0].legend(handles=[green_patch, purple_patch, blue_patch], bbox_to_anchor=(4.15, 1.46), ncol=3)

fig.supxlabel('Month', fontsize=14)
fig.supylabel('Change in monthly runoff as percent of annual runoff', fontsize=14);
fig.suptitle('Change in seasonality, 2081-2100 versus 2000-2019, {}'.format(scen), fontsize=14)
plt.tight_layout()

In [ ]:
early_C_monthmeans

## Plot all 75 basins